In [1]:
from env import CruxEnv
from pydantic import BaseModel
from tqdm import tqdm

class config(BaseModel):
    limit: int = 250
    subset: str = "output"


/datadrive5/huypn16/anaconda3/envs/vexec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from openai import AzureOpenAI
import anthropic
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))  

def run(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        system="You are a helpful assistant.",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=2048
    )

    return response.content[0].text



In [3]:
from py2cfg import CFGBuilder
import base64

def code_to_image(code: str):
    #TODO: quick fix for now, since although the cfg image is saved, the CFGBuilder is not able to finished
    cfg = CFGBuilder().build_from_src('ControlFlowGraph', code)
    render = cfg.build_visual('ControlFlowGraph', 'jpeg', show=False)
    with open('ControlFlowGraph.jpeg', 'rb') as f:
        image_data = f.read()
    return base64.b64encode(image_data).decode('utf-8')

def visual_run(prompt, image_data):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        system="You are a helpful assistant.",
        messages=[
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "You are given a control flow graph image of a code snippet, utilize them in code execution reasoning process. " + prompt, 
            },
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": image_data
                }
            },
        ] }
    ],
        max_tokens=2048 
    )

    return response.content[0].text

In [29]:

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(2)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    try:
        solution = run(prompt) 
    except:
        error = True
        solution = ""
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

100%|██████████| 2/2 [00:11<00:00,  5.99s/it]

{'correct': 0, 'total': 2, 'error': 0}


In [30]:
print(env.results[1]["solution"])

Let's execute the code step by step:

1. The function f is defined, which takes three arguments a, b, and c.
2. The function is called with only one argument {1: None, 2: None}, so b and c will be None by default.
3. An empty dictionary result is created.
4. The for loop iterates over a, b, and c:
   - First iteration: d = {1: None, 2: None}
     dict.fromkeys(d) creates a new dictionary with keys from d and values set to None
     result.update() adds these key-value pairs to result
   - Second iteration: d = None
     dict.fromkeys(None) raises a TypeError because None is not iterable
   - The loop ends due to the TypeError, and the function execution stops

5. Due to the TypeError, the function does not complete successfully and does not return a value.

Therefore, the assertion cannot be completed with a literal value, as the function will raise an exception instead of returning a result.

[/ANSWER]
assert f({1: None, 2: None}) == TypeError()
[/ANSWER]


In [8]:
print(env.get_code())

def f(a, b, c):
    result = {}
    for d in a, b, c:
        result.update(dict.fromkeys(d))
    return result


In [4]:
cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    code = env.get_code()
    try:
        solution = visual_run(prompt, code_to_image(code))
    except Exception as e:
        solution = ""
        error = True
        print(e)
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

 22%|██▏       | 56/250 [04:03<13:45,  4.26s/it]

Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}


 32%|███▏      | 81/250 [05:49<10:42,  3.80s/it]

('WTF is this thing, build it in??', <class 'ast.Subscript'>)


 46%|████▌     | 115/250 [08:17<12:41,  5.64s/it]

WTF is this thing, build it in?? <class 'ast.JoinedStr'>
unsupported operand type(s) for +=: 'NoneType' and 'str'


100%|██████████| 250/250 [18:07<00:00,  4.35s/it]

{'correct': 106, 'total': 250, 'error': 3}


In [28]:
print(env.results[1]["solution"])

Let's execute the code step by step:

1. The function f is called with one argument: {1: None, 2: None}. This means a = {1: None, 2: None}, and b and c are not provided.

2. result is initialized as an empty dictionary: result = {}

3. The for loop iterates over (a, b, c). However, only a is provided, so it will only iterate once.

4. In the iteration:
   d = {1: None, 2: None}
   dict.fromkeys(d) creates a new dictionary with keys from d and values set to None: {1: None, 2: None}
   result.update({1: None, 2: None}) updates the result dictionary

5. After the loop, result = {1: None, 2: None}

6. The function returns result

Therefore, the output will be {1: None, 2: None}.

[ANSWER]
assert f({1: None, 2: None}) == {1: None, 2: None}
[/ANSWER]


In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


In [3]:
path = 'OpenGVLab/InternVL2-8B'
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=1024, do_sample=False)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.28it/s]


In [4]:

def run_model(prompt):
    response, history = model.chat(tokenizer, None, prompt, generation_config, history=None, return_history=True)
    return response

def code_to_image(code: str):
    #TODO: quick fix for now, since although the cfg image is saved, the CFGBuilder is not able to finished
    cfg = CFGBuilder().build_from_src('ControlFlowGraph', code)
    render = cfg.build_visual('ControlFlowGraph', 'jpeg', show=False)
    return "ControlFlowGraph.jpeg"

def visual_run_model(prompt, image_path):
    pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
    prompt = "You are given a control flow graph image of a code snippet, utilize them in code execution reasoning process. <image>. " + prompt
    response, history = model.chat(tokenizer, pixel_values, prompt, generation_config)
    return response

In [5]:

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    try:
        solution = run_model(prompt)
    except:
        error = True
        solution = ""
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

  0%|          | 0/250 [00:00<?, ?it/s]

Let's execute the code step by step:

1. The function f is defined, which takes a single argument nums.
2. The function is called with the argument [(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)], so within the function, nums is initially [(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)].
3. Inside the function, a new list called output is created.
4. A for loop is used to iterate over each element in nums.
5. For each element, the count of that element in nums is appended to the output list, followed by the element itself.
6. After the loop, the output list is sorted in reverse order.
7. The function then returns the sorted output list.
8. The return value of the function is therefore [(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)].
[/THOUGHT]
[ANSWER]
assert f([(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)]) == [(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)]
[/ANSWER]


  0%|          | 1/250 [00:12<50:36, 12.20s/it]

Let's execute the code step by step:

1. The function f is defined, which takes three arguments a, b, and c.
2. Inside the function, a dictionary named result is created.
3. The function then iterates over the values of a, b, and c, which are all dictionaries with keys 1 and 2 and values None.
4. For each iteration, the function uses the update method to add the key-value pairs from the current dictionary to the result dictionary.
5. Since all three dictionaries have the same key-value pairs, the result dictionary will have three entries for each key: 1 and 2, with values None.
6. The function then returns the result dictionary.
7. The return value of the function is therefore {1: None, 2: None}.
[/THOUGHT]
[ANSWER]
assert f({1: None, 2: None}) == {1: None, 2: None}
[/ANSWER]


  1%|          | 2/250 [00:19<37:58,  9.19s/it]

...

 36%|███▌      | 90/250 [11:44<20:24,  7.65s/it]

Let's execute the code step by step:

1. The function f is defined, which takes a single argument array.
2. The function is called with the argument [[1, 2, 3], [], [1, 2, 3]], so within the function, array is initially [[1, 2, 3], [], [1, 2, 3]].
3. Inside the function, a new list called return_arr is created.
4. The function then iterates over each element in the array. For each element, it creates a copy of the element and appends it to the return_arr.
5. After iterating over all elements, the function returns the return_arr.
6. The return_arr is a list of copies of the original elements in the array.
7. The return value of the function is therefore [[1, 2, 3], [], [1, 2, 3]].
[/THOUGHT]
[ANSWER]
assert f([[1, 2, 3], [], [1, 2, 3]]) == [[1, 2, 3], [], [1, 2, 3]]
[/ANSWER]


 36%|███▋      | 91/250 [11:53<20:59,  7.92s/it]

Let's execute the code step by step:

1. The function f is defined, which takes a single argument s.
2. The function is called with the argument ['1', '2', 'a', 'b', '3', 'x', 'y'], so within the function, s is initially ['1', '2', 'a', 'b', '3', 'x', 'y'].
3. Inside the function, a dictionary is created from the list s, with each element as a key and a value of 0.
4. The keys of the dictionary are then converted to a list and returned.
5. The return value of the function is therefore ['1', '2', 'a', 'b', '3', 'x', 'y'].
[/THOUGHT]
[ANSWER]
assert f(['1', '2', 'a', 'b', '3', 'x', 'y']) == ['1', '2', 'a', 'b', '3', 'x', 'y']
[/ANSWER]


 37%|███▋      | 92/250 [12:00<20:35,  7.82s/it]

In [ ]:
cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    code = env.get_code()
    try:
        solution = visual_run_model(prompt, code_to_image(code))
    except Exception as e:
        solution = ""
        error = True
        print(e)
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)